        1. Импортируем данные. Отбрасываем столбцы, где больше половины - NaN. В остальных столбцах заменяем все NaN на среднее значение или моду в зависимости от типа данных (числа/категорийные данные). Категорийные данные конвертируем в числа через энкодинг. Затем обучаем модель с Random Forest, уровень точности измеряем через r2_score (чем ближе к 1, тем лучше).

In [1]:
import pandas as pd
val_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1030,160,RM,21.0,1680,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,118000
1,366,70,RM,59.0,10690,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,WD,Normal,147000
2,883,60,RL,NaN,9636,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,12,2009,WD,Normal,178000
3,94,190,C (all),60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2007,WD,Normal,133900
4,1360,20,RL,129.0,16737,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Normal,315000


In [2]:
columns_with_NaN_more_than_half = []
for i in train_data.columns:
    if val_data.isnull().sum()[i]>int((0.5*len(val_data))):
        columns_with_NaN_more_than_half.append(i)

In [3]:
val_data = val_data.drop(columns_with_NaN_more_than_half, axis = 1)
train_data = train_data.drop(columns_with_NaN_more_than_half, axis = 1)

In [4]:
cols = list(val_data.columns)
cols.remove("SalePrice")

In [5]:
from sklearn.impute import SimpleImputer
numeric_columns = val_data.select_dtypes(include=['number']).columns
categorical_columns = val_data.select_dtypes(include=['object']).columns

numeric_imputer = SimpleImputer(strategy='mean')
val_data[numeric_columns] = numeric_imputer.fit_transform(val_data[numeric_columns])

categorical_imputer = SimpleImputer(strategy='most_frequent')
val_data[categorical_columns] = categorical_imputer.fit_transform(val_data[categorical_columns])

numeric_imputer = SimpleImputer(strategy='mean')
train_data[numeric_columns] = numeric_imputer.fit_transform(train_data[numeric_columns])

categorical_imputer = SimpleImputer(strategy='most_frequent')
train_data[categorical_columns] = categorical_imputer.fit_transform(train_data[categorical_columns])

In [6]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
encoder = LabelEncoder()

for i in categorical_columns:
    val_data[i] = encoder.fit_transform(val_data[i])
    train_data[i] = encoder.fit_transform(train_data[i])
val_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1227.0,60.0,3,86.000000,14598.0,1,0,3,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,4,3,214000.0
1,932.0,20.0,3,70.000000,9100.0,1,3,3,0,3,...,0.0,0.0,0.0,0.0,0.0,7.0,2009.0,4,3,117500.0
2,647.0,20.0,3,60.000000,7200.0,1,3,3,0,3,...,0.0,0.0,0.0,0.0,0.0,7.0,2008.0,4,3,98300.0
3,547.0,50.0,3,70.000000,8737.0,1,0,0,0,3,...,0.0,144.0,0.0,0.0,0.0,5.0,2007.0,4,3,210000.0
4,812.0,120.0,4,69.617886,4438.0,1,3,3,0,3,...,0.0,0.0,0.0,0.0,0.0,6.0,2008.0,1,3,144500.0


In [7]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1030.0,160.0,4,21.000000,1680.0,1,3,3,0,4,...,0.0,0.0,0.0,0.0,0.0,5.0,2006.0,8,4,118000.0
1,366.0,70.0,4,59.000000,10690.0,1,3,3,0,4,...,218.0,0.0,0.0,0.0,0.0,7.0,2009.0,8,4,147000.0
2,883.0,60.0,3,70.099258,9636.0,1,0,3,0,0,...,0.0,0.0,0.0,0.0,0.0,12.0,2009.0,8,4,178000.0
3,94.0,190.0,0,60.000000,7200.0,1,3,3,0,0,...,0.0,0.0,99.0,0.0,0.0,11.0,2007.0,8,4,133900.0
4,1360.0,20.0,3,129.000000,16737.0,1,3,3,0,3,...,0.0,0.0,0.0,0.0,0.0,9.0,2006.0,8,4,315000.0


In [8]:
X_train = train_data[cols]
y_train = train_data["SalePrice"]
X_val = val_data[cols]
y_val = val_data["SalePrice"]

In [9]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators = 10, random_state=1)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_val)

In [10]:
from sklearn.metrics import r2_score

In [15]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

p = len(X_train.columns)
n = len(y_val)

mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')

r2 = r2_score(y_val, y_pred)
print(f'R-squared (R^2): {r2}')

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'Adjusted R-squared: {adjusted_r2}')

mape = np.mean(np.abs((y_val - y_pred) / y_val)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')

Mean Absolute Error (MAE): 16960.107534246577
Mean Squared Error (MSE): 504772536.0869178
Root Mean Squared Error (RMSE): 22467.14347857595
R-squared (R^2): 0.9115502014608361
Adjusted R-squared: 0.8141272349539309
Mean Absolute Percentage Error (MAPE): 9.956988223258179%


    2. Импортируем данные. Отбрасываем столбцы, где больше половины - NaN. В остальных столбцах заменяем все NaN на среднее значение или моду в зависимости от типа данных (числа/категорийные данные). Категорийные данные конвертируем в числа через энкодинг. Используя корреляцию, находим 14 наиболее коррелируемых с SalePrice столбцов. Затем обучаем модель с Random Forest, уровень точности измеряем через r2_score (чем ближе к 1, тем лучше).

In [16]:
val_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")

val_data = val_data.drop(columns_with_NaN_more_than_half, axis = 1)
train_data = train_data.drop(columns_with_NaN_more_than_half, axis = 1)

numeric_columns = val_data.select_dtypes(include=['number']).columns
categorical_columns = val_data.select_dtypes(include=['object']).columns

numeric_imputer = SimpleImputer(strategy='mean')
val_data[numeric_columns] = numeric_imputer.fit_transform(val_data[numeric_columns])

categorical_imputer = SimpleImputer(strategy='most_frequent')
val_data[categorical_columns] = categorical_imputer.fit_transform(val_data[categorical_columns])

numeric_imputer = SimpleImputer(strategy='mean')
train_data[numeric_columns] = numeric_imputer.fit_transform(train_data[numeric_columns])

categorical_imputer = SimpleImputer(strategy='most_frequent')
train_data[categorical_columns] = categorical_imputer.fit_transform(train_data[categorical_columns])

encoder = LabelEncoder()
for i in categorical_columns:
    val_data[i] = encoder.fit_transform(val_data[i])
    train_data[i] = encoder.fit_transform(train_data[i])
    
important = list(train_data.corr()["SalePrice"].sort_values(ascending=False)[1:8].index)
im = list(train_data.corr()["SalePrice"].sort_values(ascending=False)[-8:].index)
l = important + im

X_train = train_data[l]
X_val = val_data[l]

y_train = train_data["SalePrice"]
y_val = val_data["SalePrice"]

clf = RandomForestRegressor(n_estimators=10, random_state=1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)
r2_score(y_val, y_pred)

0.8758262975432859

    3. Используем предварительно обработанные данные, обучаем модель по GradientBoostingRegressor, измеряем точность по rmse.

In [17]:
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=1)
reg.fit(X_train, y_train)

y_pred = reg.predict(X_val)


p = len(X_train.columns)
n = len(y_val)

mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')

r2 = r2_score(y_val, y_pred)
print(f'R-squared (R^2): {r2}')

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'Adjusted R-squared: {adjusted_r2}')

mape = np.mean(np.abs((y_val - y_pred) / y_true)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')

Mean Absolute Error (MAE): 18417.376211798328
Mean Squared Error (MSE): 678539895.9436512
Root Mean Squared Error (RMSE): 26048.798358919576
R-squared (R^2): 0.8811014609426635
Adjusted R-squared: 0.8673823987437401
Mean Absolute Percentage Error (MAPE): 10.555189579616647%


    4. Используем предварительно обработанные данные, обучаем модель по LinearRegression, измеряем точность по rmse.

In [18]:
from sklearn.linear_model import LinearRegression

red = LinearRegression()
reg.fit(X_train, y_train)

y_pred = reg.predict(X_val)


p = len(X_train.columns)
n = len(y_val)

mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')

r2 = r2_score(y_val, y_pred)
print(f'R-squared (R^2): {r2}')

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'Adjusted R-squared: {adjusted_r2}')

mape = np.mean(np.abs((y_val - y_pred) / y_true)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')

Mean Absolute Error (MAE): 18417.376211798328
Mean Squared Error (MSE): 678539895.9436512
Root Mean Squared Error (RMSE): 26048.798358919576
R-squared (R^2): 0.8811014609426635
Adjusted R-squared: 0.8673823987437401
Mean Absolute Percentage Error (MAPE): 10.555189579616647%


    5. Используем предварительно обработанные данные, обучаем модель по Polynomial Regression, измеряем точность по rmse.

In [19]:
from sklearn.preprocessing import PolynomialFeatures

degree = 2

poly = PolynomialFeatures(degree=degree)
X_poly_train = poly.fit_transform(X_train)
X_poly_val = poly.transform(X_val)

reg = LinearRegression()
reg.fit(X_poly_train, y_train)

y_pred = reg.predict(X_poly_val)


p = len(X_train.columns)
n = len(y_val)

mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')

r2 = r2_score(y_val, y_pred)
print(f'R-squared (R^2): {r2}')

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'Adjusted R-squared: {adjusted_r2}')

mape = np.mean(np.abs((y_val - y_pred) / y_true)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')         # This model is the best -with the least r2_score

Mean Absolute Error (MAE): 17825.74345239093
Mean Squared Error (MSE): 539122142.5105008
Root Mean Squared Error (RMSE): 23219.003908662853
R-squared (R^2): 0.9055312215226282
Adjusted R-squared: 0.8946309778521623
Mean Absolute Percentage Error (MAPE): 10.625983566291088%
